In [1]:
import tensorflow as tf
import torchfile

import numpy as np
import time, sys
import os

from model import build_model

In [2]:
# constants
width = 128
loss_lambda = 0.1

In [3]:
DATA_DIR = os.getcwd().rsplit('/', 1)[0] + '/data'
checkpoint_dir = os.getcwd().rsplit('/', 1)[0] + '/checkpoint'

In [4]:
def convert_joint_to_dict(joint):
    return dict(zip(joint.name, joint.position))

def restore_vars(saver, sess, checkpoint_dir, path=None):
    """ Restore saved net, global score and step, and epsilons OR
    create checkpoint directory for later storage. """
    sess.run(tf.initialize_all_variables())

    if not os.path.exists(checkpoint_dir):
        try:
            os.makedirs(checkpoint_dir)
        except OSError:
            pass

    if path is None:
        path = tf.train.latest_checkpoint(checkpoint_dir)    
    
    if path is None:
        print ('No existing checkpoint found')
        return False
    else:
        print ('Restoring from %s' % path)
        saver.restore(sess, path)
        return True

In [5]:
# model
grasp_class_prediction, logit, grasp_image_ph, keep_prob_ph = build_model(width)
grasp_class_ph =  tf.placeholder('int64', [None])

Final shape [None, 16, 16, 32] flat_dim 8192


In [6]:
# loss
grasp_class_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=grasp_class_ph), name='grasp_class_loss')
combined_loss = grasp_class_loss

In [7]:
# evaluation
batch = 100
correct_prediction = tf.equal(tf.argmax(grasp_class_prediction, 1), grasp_class_ph)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
saver = tf.train.Saver(max_to_keep=5, keep_checkpoint_every_n_hours=1)

In [12]:
with tf.Session() as sess:
    restore_vars(saver, sess, checkpoint_dir, checkpoint_dir + '/model-39')
    val_data = torchfile.load(DATA_DIR+'/val.t7')
    n = len(val_data[b'x'][0])
    print ('%d samples' % n)
    acc = 0.
    loss = 0.
    for i in range(int(n / batch) + 1):
        start = batch * i
        if n == start:
            break
        end = min(start + batch, n)        
        rgb_image = val_data[b'x'][0][start:end].transpose(0, 2, 3, 1)
        grasp_class = val_data[b'y'][start:end] - 1
        eval_feed = {
            grasp_image_ph: rgb_image,
            grasp_class_ph: grasp_class,
            keep_prob_ph: 1.,
        }
        loss += combined_loss.eval(feed_dict=eval_feed) * (end - start)
        acc += accuracy.eval(feed_dict=eval_feed) * (end - start)
    print("Loss", loss / n)
    print("Accuracy:",acc / n)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Restoring from /home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/checkpoint/model-39
INFO:tensorflow:Restoring parameters from /home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/checkpoint/model-39
1604 samples
Loss 0.639182402011
Accuracy: 0.63466333897


In [14]:
with tf.Session() as sess:
    restore_vars(saver, sess, checkpoint_dir, checkpoint_dir + '/model-99')
    val_data = torchfile.load(DATA_DIR+'/val.t7')
    n = len(val_data[b'x'][0])
    print ('%d samples' % n)
    acc = 0.
    loss = 0.
    for i in range(int(n / batch) + 1):
        start = batch * i
        if n == start:
            break
        end = min(start + batch, n)        
        rgb_image = val_data[b'x'][0][start:end].transpose(0, 2, 3, 1)
        grasp_class = val_data[b'y'][start:end] - 1
        eval_feed = {
            grasp_image_ph: rgb_image,
            grasp_class_ph: grasp_class,
            keep_prob_ph: 1.,
        }
        loss += combined_loss.eval(feed_dict=eval_feed) * (end - start)
        acc += accuracy.eval(feed_dict=eval_feed) * (end - start)
    print("Loss", loss / n)
    print("Accuracy:",acc / n)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Restoring from /home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/checkpoint/model-99
INFO:tensorflow:Restoring parameters from /home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/checkpoint/model-99
1604 samples
Loss 0.693147420289
Accuracy: 0.36284289039


In [13]:
with tf.Session() as sess:
    restore_vars(saver, sess, checkpoint_dir, checkpoint_dir + '/model-999')
    val_data = torchfile.load(DATA_DIR+'/val.t7')
    n = len(val_data[b'x'][0])
    print ('%d samples' % n)
    acc = 0.
    loss = 0.
    for i in range(int(n / batch) + 1):
        start = batch * i
        if n == start:
            break
        end = min(start + batch, n)        
        rgb_image = val_data[b'x'][0][start:end].transpose(0, 2, 3, 1)
        grasp_class = val_data[b'y'][start:end] - 1
        eval_feed = {
            grasp_image_ph: rgb_image,
            grasp_class_ph: grasp_class,
            keep_prob_ph: 1.,
        }
        loss += combined_loss.eval(feed_dict=eval_feed) * (end - start)
        acc += accuracy.eval(feed_dict=eval_feed) * (end - start)
    print("Loss", loss / n)
    print("Accuracy:",acc / n)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Restoring from /home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/checkpoint/model-999
INFO:tensorflow:Restoring parameters from /home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/checkpoint/model-999
1604 samples
Loss 0.693147420289
Accuracy: 0.36284289039
